## Import

In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import random
import pandas as pd
import numpy as np
import os
import cv2
import math

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
import timm
from torch.optim import lr_scheduler

from tqdm import tqdm
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore')

import pickle
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import *
from sklearn.model_selection import train_test_split, KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
# import datetime, time

%matplotlib inline 
import matplotlib.pyplot as plt
import ttach as tta
import glob
from utils.early_stopping import EarlyStopping
from efficientnet_pytorch import EfficientNet
from Cream.TinyViT.models.tiny_vit import tiny_vit_21m_224, tiny_vit_21m_384


In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [4]:
CFG = {
    'IMG_SIZE':384,
    'EPOCHS':50,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':16,
    'SEED':41,
    'N_FOLD' : 5
}

## Fixed RandomSeed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Load

In [6]:
seg_img_list=glob.glob('./seg_img/*')
seg_img = pd.Series(seg_img_list) 

In [7]:
data_type=pd.Series(seg_img_list).apply(lambda x: x.split('\\')[1].split('_')[0])
seg_df=pd.concat([seg_img,data_type], 1)

In [8]:
seg_df.columns=['img_path','type']

In [9]:
train_df=seg_df[seg_df['type']=='TRAIN'].reset_index(drop=True)
test_df=seg_df[seg_df['type']=='TEST'].reset_index(drop=True)

In [10]:
train_df['img_path']=train_df['img_path'].apply(lambda x: x.replace('\\','/'))
test_df['img_path']=test_df['img_path'].apply(lambda x: x.replace('\\','/'))

In [11]:
with open('./pickle_data/Y_train_pkl','rb') as file:
        Y_train_pkl=pickle.load(file)
with open('./pickle_data/Y_test_pkl','rb') as file:
        Y_test_pkl=pickle.load(file)

In [12]:
Y_train_pkl.shape, Y_test_pkl.shape

((32994, 10), (1460, 0))

## CustomDataset

In [13]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = torch.FloatTensor(self.label_list[index])
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [14]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Cutout(num_holes=4, max_h_size=32, max_w_size=32, p=0.3),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

## Model Define

In [15]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=10):
        super(BaseModel, self).__init__()
        self.model = tiny_vit_21m_384(pretrained=True)  # backbone 모델을 tiny_vit384로 설정

        # self.model = timm.create_model('regnety_080', pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.model(x)
        x = F.sigmoid(self.classifier(x))
        return x

## Train

In [16]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [17]:
def train(model, optimizer, train_loader, val_loader, scheduler, device, nfold):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_acc = 0
    best_model = None
    # date_time = datetime.now().strftime("%m%d%H%M%S")
    total_train_loss, total_valid_loss = [],[]
    patience = 5
    early_stopping = EarlyStopping(patience = patience, path = f'./weights/best_{nfold}.pt', verbose = True)

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        train_acc =[]
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
        
            is_mixup = np.random.random() > 0.6
            if is_mixup:
                input_images, targets_a, targets_b, lam = mixup_data(imgs, labels, alpha=1.0, use_cuda=True)
                output = model(imgs)
                loss = mixup_criterion(criterion, output, targets_a, targets_b, lam)
                loss.backward()

            else:
                output = model(imgs)
                loss = criterion(output, labels)
                loss.backward()

            optimizer.step()
        
            probs  = output.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()

            train_acc.append(batch_acc)
            train_loss.append(loss.item())
                    
        _train_acc = np.mean(train_acc)
        _train_loss = np.mean(train_loss)
        
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)

        total_train_loss.append(_train_loss)
        total_valid_loss.append(_val_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Train ACC : [{_train_acc:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_loss)
        
        
        early_stopping(_val_loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    return best_model, total_train_loss, total_valid_loss

In [18]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    val_acc = []
    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            probs = model(imgs)
            
            loss = criterion(probs, labels)
            
            probs  = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()
            preds = probs > 0.5
            batch_acc = (labels == preds).mean()
            
            val_acc.append(batch_acc)
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_acc = np.mean(val_acc)


    return _val_loss, _val_acc

## Run!!

In [19]:
#strat_kfold = MultilabelStratifiedKFold(n_splits=CFG['N_FOLD'], random_state=CFG['SEED'], shuffle=True)
kfold = KFold(n_splits=CFG['N_FOLD'], random_state=CFG['SEED'], shuffle=True)

for nfold, (train_index,valid_index) in enumerate(kfold.split(train_df,Y_train_pkl)):
    
    x_train, y_train = train_df['img_path'][train_index], Y_train_pkl[train_index]
    x_valid, y_valid = train_df['img_path'][valid_index], Y_train_pkl[valid_index]
    
    x_train=x_train.reset_index(drop=True)
    x_valid=x_valid.reset_index(drop=True)

    train_dataset = CustomDataset(x_train, y_train, train_transform)
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0, pin_memory=True)

    val_dataset = CustomDataset(x_valid, y_valid, test_transform)
    val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0, pin_memory=True)

    model = BaseModel()
    model.eval()
    optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3,threshold_mode='abs',min_lr=1e-8, verbose=True)

    infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device, nfold)

    print(f'{nfold} is done')

LR SCALES: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


100%|██████████| 413/413 [00:54<00:00,  7.56it/s]


Epoch [1], Train Loss : [0.46454] Train ACC : [0.87248] Val Loss : [0.17936] Val ACC : [0.95151]
Validation loss decreased (inf --> 0.179357).  Saving model ...


100%|██████████| 413/413 [00:59<00:00,  6.98it/s]


Epoch [2], Train Loss : [0.38455] Train ACC : [0.95879] Val Loss : [0.16461] Val ACC : [0.97326]
Validation loss decreased (0.179357 --> 0.164610).  Saving model ...


...

Epoch [12], Train Loss : [0.30447] Train ACC : [0.99608] Val Loss : [0.12804] Val ACC : [0.98965]
EarlyStopping counter: 3 out of 5


100%|██████████| 413/413 [01:44<00:00,  3.97it/s]


Epoch [13], Train Loss : [0.31908] Train ACC : [0.99639] Val Loss : [0.12852] Val ACC : [0.98947]
Epoch 00013: reducing learning rate of group 0 to 3.7500e-05.
EarlyStopping counter: 4 out of 5


100%|██████████| 413/413 [01:25<00:00,  4.83it/s]

Epoch [14], Train Loss : [0.31624] Train ACC : [0.99723] Val Loss : [0.11614] Val ACC : [0.99003]
EarlyStopping counter: 5 out of 5
Early stopping
4 is done


## Inference

In [20]:
test_dataset = CustomDataset(test_df['img_path'], None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [21]:
def inference(model, test_loader, device, nfold):
    model.to(device)

    ckpt = torch.load(f'./weights/best_{nfold}.pt')
    model.load_state_dict(ckpt)

    model.eval()
    predictions = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            probs = model(imgs)

            probs  = probs.cpu().detach().numpy()
            preds = probs > 0.5
            preds = preds.astype(int)
            predictions += preds.tolist()
    return predictions

In [22]:
preds_list = [] 
for nfold in tqdm(range((CFG['N_FOLD']))):
    preds = inference(model, test_loader, device, nfold)
    preds_list.append(preds)

100%|██████████| 5/5 [01:10<00:00, 14.12s/it]


In [23]:
mean_preds = np.mean(np.stack(preds_list), axis= 0)
final_preds = (mean_preds>0.5)*1

In [24]:
preds_list

[[[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
  [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
  [0, 0, 0, 1, 1, 1, 1, 1, 0, 1],
  [1, 1, 0, 1, 1, 1, 1, 1, 1, 0],
  [0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
  [1, 1, 1, 1, 1, 0, 0, 1, 1, 0],
  [0, 0, 1, 1, 0, 1, 0, 1, 1, 1],
  [1, 0, 1, 1, 0, 1, 1, 0, 0, 1],
  [1, 1, 1, 1, 1, 0, 1, 0, 1, 1],
  [1, 0, 0, 0, 1, 0, 1, 0, 0, 0],
  [0, 0, 1, 0, 1, 0, 0, 0, 1, 1],
  [1, 0, 1, 1, 1, 0, 1, 0, 0, 0],
  [1, 1, 1, 1, 0, 0, 1, 0, 1, 1],
  [0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
  [1, 1, 1, 0, 1, 0, 0, 0, 1, 0],
  [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 1, 1, 0, 0, 0, 0, 1],
  [1, 0, 1, 1, 0, 0, 1, 1, 0, 1],
  [0, 1, 0, 1, 1, 1, 1, 0, 1, 0],
  [0, 1, 1, 0, 1, 1, 0, 1, 0, 1],
  [1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
  [1, 1, 1, 0, 1, 0, 1, 0, 0, 1],
  [0, 1, 0, 1, 1, 0, 0, 0, 0, 0],
  [1, 0, 1, 1, 0, 1, 0, 0, 0, 0],
  [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 1, 1, 1, 0, 0, 1, 1],
  [1, 0, 0, 1, 0, 0, 0, 1, 0, 0],
  [1, 1, 1, 0, 0, 1, 0, 1, 1, 1],
  [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
  [1, 0, 0, 1,

## Submission

In [25]:
submit = pd.read_csv('./sample_submission.csv')

In [26]:
submit.iloc[:,1:] = final_preds
submit.head()

,id,A,B,C,D,E,F,G,H,I,J
0,TEST_00000,0,1,0,0,0,0,1,0,1,0
1,TEST_00001,0,0,0,0,1,1,0,0,0,0
2,TEST_00002,0,0,0,0,1,1,1,1,0,1
3,TEST_00003,1,1,0,1,1,1,0,1,1,0
4,TEST_00004,0,1,0,0,1,0,0,0,0,0


In [27]:
submit.to_csv('./baseline_submit_tracer_5fold_vit_324_adamw_mixup_cutout.csv', index=False)